# EDA + Knowledge Engineering Practice – Olympics Dataset
## Part A – Basic EDA
- How many unique sports are there in the Olympics dataset?
- Which year had the maximum number of participating athletes?
- Which country has won the highest total number of medals?
- Who is the most decorated Olympic athlete (by total medals)?
- Find the trend of female athlete participation across the years.

## Part B – Intermediate EDA
- Compare Summer vs Winter Olympics in terms of total medals awarded.
- Which age group (e.g., <20, 20–30, 30–40, >40) has the highest chance of winning medals?
- What is the distribution of medals by gender?
- Which countries have dominated in Swimming and Athletics over the years?
- Find the top 10 athletes with the longest Olympic careers (span between first and last participation).

## Part C – Tricky / Knowledge Engineering Style
These require connecting multiple columns or reasoning indirectly:
- Is there a correlation between Height/Weight and medal-winning in certain sports (e.g., Basketball, Gymnastics)?
- Which countries have the highest medal-to-participation ratio (efficient performers)?
- Identify athletes who have won medals in multiple sports.
- For each host country, did their medal count improve when they hosted compared to the Olympics before/after?
- Which sports are most gender-balanced vs most gender-skewed in participation?

## Part D – Advanced Knowledge Engineering
- Using NOC mapping, group medals by continent – which continent dominates overall?
- Are team sports (e.g., Football, Hockey) more dominated by certain regions compared to individual sports?
- Which athletes participated in the most Olympics without ever winning a medal?
- Has the average age of medal winners increased or decreased over time?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Data Loading
print("LOADING THE REQUIRED DATASET AND MERGING THEM BASED ON COLUMN NOC")
athlete_df=pd.read_csv("athlete_events.csv")
noc_df=pd.read_csv("noc_regions.csv")
athlete_merged_df=athlete_df.merge(noc_df,how="inner",on="NOC")

In [ ]:
print(f"\t\t\t\tAbout Dataset\n\n\t\tShape of Dataset => {athlete_merged_df.shape}\n\n\t\tColumns of Dataset are\n\n{athlete_merged_df.columns}\n\n\t\tCount of Null\n\n{athlete_merged_df.isnull().sum()}")

In [ ]:
print(f"Information About Dataset\n")
{athlete_merged_df.info()}

In [ ]:
print("\t\t\t\tRANDOM 10 DATA FROM ATHELETE DATASET")
athlete_merged_df.sample(10)

In [ ]:
print("FILL THE NULL VALUE WITH AVERAGE VALUE IN COLUMNS { HEIGHT, WEIGHT, AGE }")
athlete_merged_df.fillna({'Height':athlete_merged_df['Height'].mean()},inplace=True)
athlete_merged_df.fillna({'Weight':athlete_merged_df['Weight'].mean()},inplace=True)
athlete_merged_df.fillna({'Age':athlete_merged_df['Age'].mean()},inplace=True)

In [ ]:
athlete_merged_df.drop(columns="notes",inplace=True)

In [ ]:
print("CHECKING FOR NEW NULL VALUES")
athlete_merged_df.isnull().sum()

### The data is cleaned

## Part A : Basic EDA

In [ ]:
# How many unique sports are there in the Olympics dataset?
sports_array=athlete_merged_df["Sport"].nunique()
print(f"No of unique sports in the Olympics dataset => {sports_array}")

In [ ]:
# Which year had the maximum number of participating athletes?
athletics_df=athlete_merged_df[athlete_merged_df["Sport"]=='Athletics']
result=athletics_df['Year'].value_counts().sort_values(ascending=False).head(1)
print("Year with the maximum number of participating Athletics \n",result)

In [ ]:
result=athlete_df.groupby("Year")["Name"].nunique().sort_values(ascending=False).head(1)
print("Year with the maximum number of participating athletes \n",result)

In [ ]:
# Which country has won the highest total number of medals?
result=athlete_merged_df.groupby('region')['Medal'].count().sort_values(ascending=False).head(1)
print("country that has won the highest total number of medals are \n",result)

In [ ]:
# Who is the most decorated Olympic athlete (by total medals)?
print("the most decorated Olympic athlete (by total medals)")
athlete_merged_df.groupby("Name")['Medal'].value_counts().sort_values(ascending=False).unstack(fill_value=0).head(1)

In [ ]:
# Find the trend of female athlete participation across the years.
female_df=athlete_merged_df[athlete_merged_df['Sex']=='F'].drop_duplicates(["Name","Year"])
count=female_df.groupby("Year")["Name"].nunique()

plt.plot(count.index,count.values,marker='o',color='purple',label="Female Athletes (count)")
plt.title("Trend of Female Athlete Participation in the Olympics")
plt.legend()
plt.xlabel("year")
plt.ylabel("Count")
plt.grid(color="gray",linestyle="--")
plt.show()

## Part B : Intermediate EDA

In [ ]:
# Compare Summer vs Winter Olympics in terms of total medals awarded
print("Summer vs Winter Olympics in terms of total medals awarded")
athlete_merged_df.dropna(subset='Medal').groupby("Season")["Medal"].count()

In [ ]:
# Which age group (<20, 20–30, 30–40, >40) has the highest chance of winning medals?
athlete_merged_df['Age-Group']=pd.cut(athlete_merged_df["Age"],bins=[0,20,30,40,100],labels=['<20', '20–30', '30–40', '>40'])
medal_age_grp=athlete_merged_df.dropna(subset='Medal').groupby("Age-Group",observed=True)['Name'].count()
print("\nMedals By Age Group\n",medal_age_grp)
tot_part_age_grp=athlete_merged_df.groupby("Age-Group",observed=True)["Name"].count()
print("\nTotal Participation By Age Group\n",tot_part_age_grp)
chance=round((medal_age_grp/tot_part_age_grp)*100,2).reset_index().max()
print("\nAge Group that has the highest chance of winning medals is\n",chance)

In [ ]:
# What is the distribution of medals by gender?
print("The distribution of medals by gender")
athlete_merged_df.dropna(subset="Medal").groupby("Sex")["Medal"].count()

In [ ]:
# Which countries have dominated in Swimming and Athletics over the years?
athlete_merged_df.dropna(subset='Medal').query("Sport in ['Swimming' , 'Athletics']").groupby(["region",'Sport'])["Medal"].count().sort_values(ascending=False).head(10)

In [ ]:
# Top 10 athletes with the longest Olympic careers (span between first and last participation)
career_span=athlete_merged_df.groupby("Name")["Year"].agg(['max','min'])
career_span["Span"]=career_span["max"]-career_span['min']
print("Top 10 athletes with the longest Olympic careers\n")
career_span.sort_values("Span",ascending=False).head(10)

## Part C : Tricky / Knowledge Engineering Style

In [ ]:
# Is there a correlation between Height/Weight and medal-winning in certain sports (e.g., Basketball, Gymnastics)?
subset=athlete_merged_df.query('Sport in [\'Basketball\']')
medal_df=subset[subset["Medal"].notna()]
non_medal_df=subset[subset["Medal"].isna()]
fig,ax=plt.subplots(1,2,figsize=(10,5))
fig.supxlabel("Height")
fig.supylabel("weight")
ax[0].scatter(medal_df['Height'],medal_df['Weight'],c="red",edgecolors="black",alpha=0.3,label="Medalist")
ax[0].set_title("Height VS Weight Of Medalist")
ax[0].legend()
ax[1].scatter(non_medal_df['Height'],non_medal_df['Weight'],c="yellow",edgecolors="black",alpha=0.7,label="Non-Medalist")
ax[1].set_title("Height VS Weight Of Non-Medalist")
ax[1].legend()
plt.tight_layout()
plt.show()
print("Yes, there a correlation between Height/Weight and medal-winning in certain sports")

In [ ]:
#  Which countries have the highest medal-to-participation ratio (efficient performers)?
total_participation_per_country=athlete_merged_df.groupby("region")["Name"].nunique()
total_medal_per_country=athlete_merged_df.dropna(subset="Medal").groupby("region")["Medal"].count()
print("Countries that have the highest medal-to-participation ratio (efficient performers) are")
(total_medal_per_country/total_participation_per_country).sort_values(ascending=False).head(10).index

In [ ]:
# Identify athletes who have won medals in multiple sports.
athletes_with_medal=athlete_merged_df.dropna(subset="Medal").groupby("Name")["Sport"].nunique()
result=athletes_with_medal[athletes_with_medal>1].sort_values(ascending=False).head(1)
print("Athletes who have won medals in multiple sports \n",result)

In [ ]:
# For each host country, did their medal count improve when they hosted compared to the Olympics before/after?
# Medal counts by year and country
medal_per_year=athlete_merged_df.dropna(subset="Medal").groupby(['Year','region'])['Medal'].count().reset_index()
# Host years and countries
hosts=athlete_merged_df.drop_duplicates(subset=['Year','City'])[['Year','region','City']].dropna()
medal_per_year.merge(hosts,on=['Year','region'],how="inner").sort_values("region")

In [ ]:
# Which sports are most gender-balanced vs most gender-skewed in participation?
gender_counts=athlete_merged_df.groupby(["Sport","Sex"])["Name"].nunique().unstack(fill_value=0)
gender_counts["Gender-Ratio"]=gender_counts.min(axis=1)/gender_counts.max(axis=1)
gender_balanced=gender_counts.sort_values("Gender-Ratio",ascending=False).head(10)
gender_skewed=gender_counts.sort_values("Gender-Ratio").head(10)
print("Sports are most gender-balanced in participation\n\n",gender_balanced)
print("\nSports are most gender-skewed in participation\n\n",gender_skewed)

## Part D : Advanced Knowledge Engineering


In [ ]:
# Using NOC mapping, group medals by continent – which continent dominates overall?
res=athlete_merged_df.dropna(subset="Medal").groupby("region")["Medal"].count()
print(f"{res.idxmax()} dominates overall with count {res.max()}")

In [ ]:
# Are team sports (e.g., Football, Hockey) more dominated by certain regions compared to individual sports?
team_sports = [
    "Basketball","Football","Tug-Of-War","Ice Hockey","Handball",
    "Water Polo","Hockey","Rowing","Softball","Volleyball",
    "Synchronized Swimming","Baseball","Rugby Sevens","Beach Volleyball",
    "Curling","Rugby","Lacrosse","Polo","Cricket","Military Ski Patrol"
]
athlete_merged_df["Sport_Type"]=athlete_merged_df['Sport'].apply(lambda x : "Team" if x in team_sports else "Individual")
team_country=athlete_merged_df.groupby(["region",'Sport_Type'])["Medal"].count().unstack(fill_value=0).sort_values(by="Team",ascending=False).head(5)
plt.plot(team_country.index,team_country.Individual,marker='o',color="green",label="Individual Sport")
plt.plot(team_country.index,team_country.Team,marker='o',color="brown",label="Team Sport")
plt.legend()
plt.title("Team VS Individual Medal Count per Region")
plt.xlabel("Region")
plt.ylabel("Medal Count")
plt.show()
print("Individual sports contribute the majority of medals overall, while team sports are dominated by a smaller set of regions (e.g., USA, Russia, Germany, Canada).")

In [ ]:
# Which athletes participated in the most Olympics without ever winning a medal?
athletes_list=athlete_merged_df[athlete_merged_df['Medal'].isna()].groupby('Name')['Year'].nunique().sort_values(ascending=False).head(10)
print("Athletes that participated in the most Olympics without ever winning a medal are")
athletes_list

In [ ]:
# Has the average age of medal winners increased or decreased over time?
athlete_merged_df['Age']=round(athlete_merged_df["Age"])
atheletes_list=athlete_merged_df.dropna(subset='Medal').groupby(["Year"])["Age"].mean()
plt.plot(atheletes_list.index,atheletes_list.values,marker="o",color="purple")
plt.xlabel("Year")
plt.ylabel("Average Age")
plt.title("Average Age Over Time")
plt.show()
if atheletes_list.iloc[-1] > atheletes_list.iloc[0]:
    print("The average age of medal winners has increased over time")
else:
    print("The average age of medal winners has decreased over time")

In [ ]:
print("Saving Cleaned Dataset")
athlete_merged_df.to_csv("Full_cleaned_Olympics_DS_1896-2016.csv")